## Test data preparation

In [9]:
import os

In [2]:
%pwd

'c:\\Tom\\HKA\\7_Semester\\Domänenprojekt_2\\DoPro'

In [3]:
os.chdir("../")

In [3]:
%pwd

'c:\\Tom\\HKA\\7_Semester\\Domänenprojekt_2\\DoPro'

### Data Preparation config class (entity)

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreparationConfig:
    """Entity-Class for data preparation config params."""

    root_dir: Path
    """Directory into which data will be loaded."""

    weather_data_path: Path
    """Directory where raw weather data is stored."""

    energy_data_path: Path
    """Directory where raw energy data is stored."""

    training_data_path: Path
    """Directory into which training data will be loaded."""

    test_data_path: Path
    """Directory into which test data will be loaded."""

### update config manager class

In [3]:
from src.dopro2_HEFTcom_challenge.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
import yaml
from loguru import logger

In [4]:
class ConfigurationManager:
    """Class to manage all configurations."""

    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ) -> None:
        """
        Constructor for ConfigurationManager Class.
        Creates artifacts folder.

        :param config_filepath: Path to config.yaml file
        :param params_filepath: Path to params.yaml file

        """
        with config_filepath.open("r") as f:
            self.config: dict = yaml.safe_load(f)

        # with params_filepath.open("r") as f:
        #     self.params: dict = yaml.safe_load(f)

        os.makedirs(self.config["artifacts_root"], exist_ok=True)
        logger.info("created directory at: {}", self.config["artifacts_root"])

    def get_data_preparation_config(self) -> DataPreparationConfig:
        """
        Get all config params and create folder in artifacts dir.
        
        :return: values from config.yaml
        :rtype: DataPreparationConfig
        """
        config = self.config["data_preparation"]

        os.makedirs(config["root_dir"], exist_ok=True)
        logger.info("created directory at: {}", config["root_dir"])

        data_preparation_config = DataPreparationConfig(
            root_dir=config["root_dir"],
            weather_data_path=config["weather_data_path"],
            energy_data_path=config["energy_data_path"],
            training_data_path=config["training_data_path"],
            test_data_path=config["test_data_path"]
        )

        return data_preparation_config

### create data preparation component

In [5]:
import pandas as pd
import xarray as xr

In [19]:
class DataPreparation:
    """Class to performe data preparation."""

    def __init__(self, config: DataPreparationConfig) -> None:
        """
        Constructor for DataPreparation class.

        :param config: config values from config.yaml
        """

        self.config = config

    def cleaning_energy_data(self) -> None:
        # TODO: handling missing values, outliers, inconsistencies
        logger.info("Start cleaning energy data")
        energy_files = Path(self.config.energy_data_path).glob("*.csv")
        df_raw = pd.concat(
            (pd.read_csv(f) for _, f in enumerate(energy_files)), 
            ignore_index=True
        )
        df = (df_raw
            .assign(dtm=pd.to_datetime(df_raw["dtm"]),
                    Wind_MWh_credit=0.5*df_raw["Wind_MW"] - df_raw["boa_MWh"],
                    Solar_MWh_credit=0.5*df_raw["Solar_MW"]
                    )
        )
        # TODO: Split into wind and solar df
        df.to_parquet(f"{self.config.root_dir}/energy_processed.parquet")
        logger.info("Cleaned energy data: file safed under {}", self.config.root_dir)



    def transform_data(self):
        ...
        # TODO: feature scaling, encoding, ...

    # def reduce_data(self):

    def splitting_data(self):
        ...
        # TODO: training, validation, test sets

In [20]:
try:
    config = ConfigurationManager()
    data_preparation_config = config.get_data_preparation_config()
    data_preparation = DataPreparation(config=data_preparation_config)
    data_preparation.cleaning_energy_data()
except Exception as e:
    raise e

2024-10-08 23:19:07.177 | INFO     | __main__:__init__:24 - created directory at: artifacts
2024-10-08 23:19:07.179 | INFO     | __main__:get_data_preparation_config:36 - created directory at: artifacts/prepared_data
2024-10-08 23:19:07.180 | INFO     | __main__:cleaning_energy_data:15 - Start cleaning energy data
2024-10-08 23:19:07.708 | INFO     | __main__:cleaning_energy_data:28 - Cleaned energy data: file safed under artifacts/prepared_data
